In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import string
import os
import pandas as pd
import datetime
import numpy as np
import statsmodels.api as sm
import math
from scipy.linalg import svd
import scipy.linalg as la
import scipy.sparse.linalg as sla

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
mu=0.2
tol=1e-10

In [ ]:
def sq(a,b,step):
    r = a
    new = a
    for i in range(10000):
        new = new + step
        if new <= b:
            r = r+ new
        else:
            break
    return r

In [ ]:
def soft_threshodr(groups,nc,w,mu):
    return w/(1+mu)

In [ ]:
def soft_threshodl(groups,nc,w,mu):
    val =[float(np.sign(w1)*np.max(np.abs(w1)-mu,0)) for w1 in w]
    return np.array(val).reshape((len(val),1))

In [ ]:
def soft_threshode(groups,nc,w,mu):
    val =[float(np.sign(w1)*np.max(np.abs(w1)-0.5*mu,0))/(1+0.5*mu) for w1 in w]
    return np.array(val).reshape((len(val),1))

In [ ]:
def soft_threshodg(groups,nc,w,mu):
    w1 = w
    for i in range(nc):
        ind = (groups == i)
        wg = w1[ind,:]
        nn= np.size(wg)
        n2 = math.sqrt(sum(pow(wg,2)))
        if n2 < mu :
            w1[ind,:] = zeros(nn,1)
        else:
            w1[ind,:] = wg - mu*wg/n2

    return w1

In [ ]:
def lossh(y,yhat,mu):
    r= abs(yhat['RET']-y)
    l= np.zeros(len(r))
    ind = (r>mu)
    l[ind] = 2*mu*r[ind]-mu*mu
    ind = (r<= mu)
    l[ind]  = r[ind]*r[ind]
    return np.mean(l)

In [ ]:
def f_gradh(w,X,y,mu):
    r = np.matmul(X,w)-y['RET']
    ind0 = np.where(abs(r)<= mu)
    ind0 = pd.Series(np.asarray(ind0)[0])
    ind1 = np.where(r>mu)
    ind1 = pd.Series(np.asarray(ind1)[0])

    indf1 = np.where(r< (-mu))
    indf1 = pd.Series(np.array(indf1)[0])
    grad = np.matmul(np.transpose(X.iloc[ind0,:]),(np.matmul(X.iloc[ind0,:],w)-y.iloc[ind0,0]))+np.matmul(np.transpose(mu*X.iloc[ind1,:]),np.ones(len(ind1)))-np.matmul(np.transpose(mu*X.iloc[indf1,:]),np.ones(len(indf1)))
    return grad

In [ ]:
def proximalH_l(groups,nc,xtest,mtrain,ytest,w,X,y,mu,tol,L,l2,func):
    w= np.asarray(w[0])
    dim = X.shape[0]
    max_iter = 3000
    gamma= 1/L
    l1 = l2

    v= w
    yhatbig1=np.matmul(xtest,w) + mtrain.values
    r20=lossh(yhatbig1,ytest,mu)
    for t in range(max_iter):
        vold=v
        w_perv=w
        w= np.asarray(v-(gamma*f_gradh(v,X,y,mu)))
        mu1=l1*gamma
        w=func(groups,nc,w,mu1)
        w = np.asarray([float(w1) for w1 in w])
        v=w + t/(t+3)*(w-w_perv)
        if ((np.linalg.norm(v-vold)**2) < ((np.linalg.norm(vold)**2)*tol) or np.sum(abs(v-vold))==0):
            break
    return v

In [ ]:
X_all = pd.read_csv('../input/project2/X_cleaned.csv')
Y_all = pd.read_csv('../input/project2/Y_cleaned.csv')
Date_index = pd.read_csv('../input/project2/Date_list.csv')
Date_index['DATE']=pd.to_datetime(Date_index['DATE'])

In [ ]:
nums= 30
start_dates = [pd.to_datetime('1957-03-01')+pd.offsets.DateOffset(years=x) for x in range(nums) ]

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
for m in range(len(start_dates)):

    start_date = start_dates[m]

    end_date =  start_date + pd.offsets.DateOffset(years=18)

    ind_training = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)

    ytrain = Y_all[ind_training]

    xtrain = X_all[ind_training]

    ####for validation

    start_date =  end_date
    end_date = start_date + pd.offsets.DateOffset(years=12)

    ind_vali = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)

    ytest = Y_all[ind_vali]
    xtest= X_all[ind_vali]

    #####for test  oos

    start_date =  end_date
    end_date = start_date + pd.offsets.DateOffset(years=1)
    ind_test = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)
    #print(type(ind_test))
    yoos = Y_all[ind_test]
    xoos= X_all[ind_test]

    #######Monthly Demean %%%
    ytrain_demean= ytrain - np.mean(ytrain);
    ytest_demean=ytest-np.mean(ytest);
    mtrain=np.mean(ytrain);
    mtest=np.mean(ytest);

    sd=np.zeros(len(xtrain.columns)) #dim of sd? sd for each characteristics
    for i in range(len(xtrain.columns)):
        s=np.std(xtrain.iloc[:,[i]])
        colnames = xtrain.columns
        if s.values > 0:
            colname = colnames[i]
            xtrain.loc[:,colname]=xtrain[colname]/s.values
            xtest.loc[:,colname]=xtest.loc[:,colname]/s.values
            xoos.loc[:,colname]=xoos.loc[:,colname]/s.values
            sd[i]=s.values
    XX =np.matmul(np.transpose(xtrain),xtrain)
    U,S,V=svd(XX)
    #### singular value decomposition such that XX=U*S*V'
    L=S[0]
    ###disp 'Lasso L = '
    ####disp(L)
    Y=ytrain_demean;
    XY= np.matmul(np.transpose(xtrain),Y)
    groups=0
    nc=0
            
    #### Start to Train %%%
   
    lamv=np.arange(-2,4,0.1)
    alpha=1
    r=np.zeros((3,len(lamv)))
    y_ret=ytest['RET'].reset_index(drop=True)
    y_ret1=ytrain['RET'].reset_index(drop=True)
    y_ret2=yoos['RET'].reset_index(drop=True)


    for j in range(len(lamv)):
        l2=10**lamv[j]
        ridge = Ridge(alpha=l2)
        yhatres= ridge.fit(xtrain,ytrain_demean)
        yhatbig1= yhatres.predict(xtest)+ mtrain.values
        yhatbig1=pd.Series(yhatbig1.flatten())
        #prediction for oos
        r[0,j]=1-sum((yhatbig1-y_ret)**2)/sum((y_ret-mtrain.values)**2) # oos r2
        yhatbig1=yhatres.predict(xtrain) + mtrain.values
        yhatbig1=pd.Series(yhatbig1.flatten())
        r[1,j]=1-(yhatbig1-y_ret1).pow(2).sum()/sum(pow(y_ret1-mtrain.values,2))
        yhatbig1=yhatres.predict(xoos) + mtrain.values
        yhatbig1=pd.Series(yhatbig1.flatten())
        r[2,j]=1-(yhatbig1-y_ret2).pow(2).sum()/sum(pow(y_ret1-mtrain.values,2))
        


    r2_oos=r[2,np.argmax(r[0,:])]
    r2_is=r[1,np.argmax(r[0,:])]
    l2=10**lamv[np.argmax(r[0,:])]
    ridge = Ridge(alpha=l2)
    yhatres= ridge.fit(xtrain,ytrain_demean)
    b=np.transpose(pd.DataFrame(yhatres.coef_))

    print('Ridge R2 : '+np.str0(r2_oos))
    print('Ridge R2_is : '+np.str0(r2_is))
    
    b=proximalH_l(groups,nc,xtest,mtrain,ytest,b,xtrain,ytrain_demean,mu,tol,L,l2,soft_threshodr)
    yhatbig1 = np.matmul(xoos,b)+mtrain.values
    r2_oos=1- np.linalg.norm(yhatbig1-yoos['RET'])**2 / np.linalg.norm((yoos['RET']-mtrain.values))**2
    yhatbig1 = np.matmul(xtrain,b) + mtrain.values
    r2_is = 1 - np.linalg.norm(yhatbig1-ytrain['RET'])**2 / np.linalg.norm((ytrain['RET']-mtrain.values))**2
    b=pd.DataFrame(b)
    print('Ridge+H R2 : '+np.str0(r2_oos) )


<!--
Ridge R2 : -19.034196220925942
Ridge R2_is : 0.035349434130731106
Ridge+H R2 : -17.501737814113433
Ridge R2 : -12.910768168878079
Ridge R2_is : 0.03527324928405795
Ridge+H R2 : -35.98689618678933
Ridge R2 : -17.05457450724823
Ridge R2_is : 0.028753100845712365
Ridge+H R2 : -57.43926042959815
Ridge R2 : -2.9969986169883276
Ridge R2_is : 0.023825688049056604
Ridge+H R2 : -7.577399616660996
Ridge R2 : -4.196540033700964
Ridge R2_is : 0.02361850774258789
Ridge+H R2 : -9.23062135162839
Ridge R2 : -0.3583109123851591
Ridge R2_is : 0.023038316068705322
Ridge+H R2 : -7.800839536986771
Ridge R2 : 0.36521100664056827
Ridge R2_is : 0.031951522800620746
Ridge+H R2 : -4.156820771943164
Ridge R2 : 0.6750825428874229
Ridge R2_is : 0.029658242640348775
Ridge+H R2 : -1.6698026266497838
Ridge R2 : -14.71290112464295
Ridge R2_is : 0.022469555546845155
Ridge+H R2 : -183.53292503680217
Ridge R2 : -38.13920889618736
Ridge R2_is : 0.021034908350779702
Ridge+H R2 : -445.08845639123405
Ridge R2 : -17.937382706911965
Ridge R2_is : 0.02102078296773946
Ridge+H R2 : -155.04078503084432
Ridge R2 : -10.079618144023074
Ridge R2_is : 0.018501208664726043
Ridge+H R2 : -51.05704837307169
Ridge R2 : 0.710433716936004
Ridge R2_is : 0.022749367298982692
Ridge+H R2 : -0.039412394762196046
Ridge R2 : 0.7424448138074125
Ridge R2_is : 0.01642371019805966
Ridge+H R2 : 0.01827194765815965
Ridge R2 : 0.7784773753024488
Ridge R2_is : 0.01884469362071084
Ridge+H R2 : -0.2858895957968526
Ridge R2 : 0.8751263376960323
Ridge R2_is : 0.013599409301093601
Ridge+H R2 : -0.10174897865962862
Ridge R2 : 0.9201239391760959
Ridge R2_is : 0.01211348314127525
Ridge+H R2 : -0.17854314270278904
Ridge R2 : 0.9665378020714807
Ridge R2_is : 0.013849495393693845
Ridge+H R2 : -0.039202535946227135
Ridge R2 : 0.9648804921962102
Ridge R2_is : 0.013114213348095705
Ridge+H R2 : -0.07070048234214754
Ridge R2 : 0.9665688852274836
Ridge R2_is : 0.0113891693283098
Ridge+H R2 : -0.007943495660429845
Ridge R2 : 0.9669171056715198
Ridge R2_is : 0.01109225389038515
Ridge+H R2 : 0.03663313848262573
Ridge R2 : 0.9369748072345325
Ridge R2_is : 0.011403164414350409
Ridge+H R2 : 0.059553202076599865
Ridge R2 : 0.9239101128729279
Ridge R2_is : 0.010663564112960322
Ridge+H R2 : -0.05637369731097408
Ridge R2 : 0.9769584754262143
Ridge R2_is : 0.010332686539059721
Ridge+H R2 : -0.014766029696281047
Ridge R2 : 0.9682478017024312
Ridge R2_is : 0.009582509705564402
Ridge+H R2 : 0.006587775212602676
Ridge R2 : 0.9798597771906581
Ridge R2_is : 0.01176364635870264
Ridge+H R2 : -0.022830398601165758
Ridge R2 : 0.9824622755382778
Ridge R2_is : 0.010234203762411131
Ridge+H R2 : -0.005372790628242585
Ridge R2 : 0.9860036244786758
Ridge R2_is : 0.009885420848224191
Ridge+H R2 : -0.006925459699056891
Ridge R2 : 0.9822007192460156
Ridge R2_is : 0.009503454580352733
Ridge+H R2 : 0.02226009659137107
Ridge R2 : 0.9809069700044362
Ridge R2_is : 0.010652251952294689
Ridge+H R2 : -0.02067784062624356